# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#loading csv file
data_path = "../output_data/Cities_pd.csv"

weatherdata_df = pd.read_csv(data_path)

weatherdata_df.head()


,Unnamed: 0,Lat,Long,City,Country,Date,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,-34.261312,-0.584121,luderitz,na,1.601704e+09,13.54,74.0,0.0,2.14
1,1,4.030846,-19.262935,goderich,sl,1.601704e+09,8.33,69.0,95.0,0.45
2,3,77.515467,173.190307,pevek,ru,1.601704e+09,0.49,87.0,96.0,1.32
3,4,14.609636,15.299001,mao,td,1.601704e+09,16.00,55.0,20.0,3.60
4,5,4.124654,138.969428,airai,pw,1.601704e+09,25.83,43.0,55.0,2.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = weatherdata_df[["Lat", "Long"]]
humidity = weatherdata_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# narrow down the cities with wind speed less than 10 mph, no clouds and temp between 20 and 40
cond1 = weatherdata_df["Wind Speed"] <= 10
cond2 = weatherdata_df["Cloudiness"] == 0
cond3 = weatherdata_df["Max Temperature"] >= 20
cond4 = weatherdata_df["Max Temperature"] <= 40

narrowdf = weatherdata_df[cond1 & cond2 & cond3 & cond4].dropna()
narrowdf.count()


Unnamed: 0         48
Lat                48
Long               48
City               48
Country            48
Date               48
Max Temperature    48
Humidity           48
Cloudiness         48
Wind Speed         48
dtype: int64

In [6]:
hotel_df = narrowdf.loc[:,["City","Country", "Lat", "Long"]]
hotel_df["Hotel Name"] = ""
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    city_name = row["City"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Logding" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        print(f"Processing Results for: {city_name}.")
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Processing Results for: bardiyah.
Missing field/result... skipping.
Processing Results for: gazli.
Missing field/result... skipping.
Processing Results for: karratha.
Missing field/result... skipping.
Processing Results for: riyadh.
Missing field/result... skipping.
Processing Results for: joshimath.
Missing field/result... skipping.
Processing Results for: arraial do cabo.
Missing field/result... skipping.
Processing Results for: nouadhibou.
Missing field/result... skipping.
Processing Results for: cabo san lucas.
Missing field/result... skipping.
Processing Results for: yulara.
Missing field/result... skipping.
Processing Results for: atar.
Missing field/result... skipping.
Processing Results for: santa maria da vitoria.
Missing field/result... skipping.
Processing Results for: poros.
Missing field/result... skipping.
Processing Results for: roebourne.
Missing field/result... skipping.
Processing Results for: sakakah.
Missing field/result... skipping.
Processing Results for: marzuq.


In [7]:
hotel_df

,City,Country,Lat,Long,Hotel Name
13,bardiyah,ly,30.007429,25.049149,
27,gazli,uz,42.551602,64.088470,
37,karratha,au,-20.743087,114.605621,
58,riyadh,sa,21.058468,46.616881,
75,joshimath,in,34.057527,81.994482,
85,arraial do cabo,br,-36.632874,-23.988634,
117,nouadhibou,mr,22.183719,-22.458374,
141,cabo san lucas,mx,14.598167,-117.959704,
164,yulara,au,-22.201737,129.104461,
172,atar,mr,19.820187,-9.868975,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))